<a href="https://colab.research.google.com/github/CSwan300/Basic_Job_Search_WebSite/blob/main/CW1EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**
---
## **Dataset:**
---
> The dataset represents country-level measures of self-reported life satisfaction (the "Ladder score") alongside socioeconomic, social-capital, health, freedom, generosity, and corruption indicators used to contextualize why some countries report higher happiness than others. It is relevant because it enables exploratory analysis of how factors like income, social support, and institutional trust relate to national well-being, directly supporting the coursework’s goals to interpret real-world data, demonstrate cleansing and transformation, and derivation off insights from statistical relationships
or complete an EDA.
---
## **Common Real-World Data Issues found in this dataset:**
---
>
>
**Missing or null values:**
>
>Some rows have NaN in fields such as Perceptions of corruption and Dystopia + residual, which require a clear handling strategy (drop, impute, or flag).

**Unnamed or irrelevant columns:**
> An Unnamed 0 column appears in the CSV and should be verified as non-informative and removed or explained accordingly as part of cleaning.

**Duplicate entries:**
>The dataset can include duplicated rows or partially duplicated records that must be detected and resolved using transparent criteria to prevent bias in analysis.

**Inconsistent formats and types:**
>Numeric fields may be read as strings, or spacing and naming inconsistencies can exist; ensure data types are corrected and column names standardized before analysis.

**Outliers and unusual values:**
>Some numeric variables may contain extreme values that influence distributions and correlations; flag using a 3-standard-deviation rule and decide whether to retain with explanation or mitigate their influence.

**Measurement noise and residual structure:**
>The Dystopia + residual term captures unexplained variance and model baseline, reminding that not all variation is accounted for by observed variables and that simple correlations may omit confounders.

**Structural and scope differences:**
>Country-year data may not be complete for all countries or years, leading to imbalance that affects comparisons and any temporal interpretation; these gaps should be noted and dealt with explicitly.

**Potential data entry errors:**
>Impossible or contradictory values (e.g., negative scores or implausible ranges) and correct or exclude as warranted, documenting all decisions for transparency.

**Contextual ambiguity:**
> Some features require domain definitions (e.g., how Social support is measured); use the provided features document to anchor interpretations and avoid mislabelling constructs.

**Reproducibility and documentation:**
> Every change-dropping columns, imputing, deduplicating—should be recorded with code and rationale to meet assessment standards on clarity and justification.
---
## **What will my EDA Explore**
---

 > MY EDA will attempt to answer these questions and hypostasis

1.   >  Higher Log GDP per capita and stronger Social support are positively associated with higher Ladder scores across countrie?
>
2.   >How do countries group across low, medium, and high happiness bands, and which factors most differentiate these groups?
>
3.   >Which variables show the strongest association with national life satisfaction (Ladder score), and are these relationships positive or negative?



# Data Loading and initial inspection

In [94]:
#used for formatiing
formatline = "-" * 75

#only run the first time colab is booted up on an instance
from google.colab import drive
#checks if it is mounted

#drive.mount('/content/drive')
#Import the csv file
#path to the csv file in google drive (i stored it in a folder called coursework)
csv_path = "/content/drive/MyDrive/Coursework/Happiness.csv"
#import lines
import pandas as pd
#obtain dataframe
df = pd.read_csv(csv_path)

# rows and columns and display
rows, cols = df.shape
print(f"Rows: {rows}, Columns: {cols}")
print(formatline)
print(f"All Datatypes:\n{formatline} \n{df.dtypes}")
#Display the first 5
print(formatline)
print("first 5 Entries")
print(formatline)
df.head(5)


Rows: 148, Columns: 17
---------------------------------------------------------------------------
All Datatypes:
--------------------------------------------------------------------------- 
Year                              int64
Rank                              int64
Country name                     object
Ladder score                    float64
Unnamed 0                       float64
Log GDP per capita              float64
Social support                  float64
Healthy life expectancy         float64
Freedom to make life choices    float64
Generosity                      float64
Perceptions of corruption       float64
Dystopia + residual             float64
Unnamed: 12                     float64
Unnamed: 13                     float64
Unnamed: 14                     float64
Unnamed: 15                      object
Unnamed: 16                      object
dtype: object
---------------------------------------------------------------------------
first 5 Entries
-----------------------

,Year,Rank,Country name,Ladder score,Unnamed 0,Log GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Dystopia + residual,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,2024,1,Finland,7.736,7.810,1.749,1.783,0.824,0.986,0.110,0.502,1.782,NaN,NaN,NaN,NaN,NaN
1,2024,2,Denmark,7.521,7.611,1.825,1.748,0.820,0.955,0.150,0.488,1.535,NaN,NaN,NaN,NaN,NaN
2,2024,3,Iceland,7.515,7.606,1.799,1.840,0.873,0.971,0.201,0.173,1.659,NaN,NaN,NaN,NaN,NaN
3,2024,4,Sweden,7.345,7.427,1.783,1.698,0.889,0.952,0.170,0.467,1.385,NaN,NaN,NaN,NaN,NaN
4,2024,5,Netherlands,7.306,7.372,1.822,1.667,0.844,0.860,0.186,0.344,1.583,NaN,NaN,NaN,NaN,NaN


#Data Cleansing and pre processing

## **Indentification off issues**
**Unnamed or irrelevant columns:**
>Unnamed and several trailing empty columns are present and should be dropped to avoid confusion during analysis.

**Missing values:**
>Some rows (e.g., Canada, Slovenia, Czechia, UAE, Germany, El Salvador) show blanks for "Ran"” or "Ladder score" variants in the snippet; these should be assessed and handled via imputation or removal depending on analytic goals.

**Duplicates:**
> Country-year should be unique; check and drop exact duplicates if any exist to prevent bias in summaries.

**Obvious errors:**
> Minor formatting issues like extra spaces in the Austria row (comma-space in the line) and potential type inference problems may occur.

**Outliers:**
> Within numerical columns such as Ladder score, Log GDP per capita, Social support, Healthy life expectancy, Freedom to make life choices, Generosity, Perceptions of corruption, and Dystopia + residual with a 3 standard deviation threshold to flag unusual entries for further review (store in a seperate dataframe) rather than automatic removal.



In [97]:
#code for cleaning the dataframe
import pandas as pd
import numpy as np
# 1) Remove columns like "Unnamed: 0", "Unnamed: 1", etc.
df = df.loc[:, ~df.columns.str.lower().str.startswith('unnamed')]

# 2) Remove columns that are entirely empty (every cell is NaN)
df = df.dropna(axis=1, how='all')

# 3) Remove duplicates
df = df.drop_duplicates(ignore_index=True)

# 4) Remove exact duplicate rows
df = df.drop_duplicates(ignore_index=True)

# 5) Convert key numeric columns safely (non-numeric becomes NaN)
for col in ['Year', 'Rank', 'Ladder score']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 6) Try to convert other columns to numeric if they look like numbers
#    If conversion fails, keep the original text values.
protect = {'Country name', 'Year', 'Rank', 'Ladder score'}
for col in df.columns:
    if col in protect:
        continue
    try:
        df[col] = pd.to_numeric(df[col], errors='raise')
    except Exception:
        # Not a purely numeric column; leave it as text
        pass

# 7) Make sure the key fields are present and not missing
required_cols = [c for c in ['Year', 'Rank', 'Country name', 'Ladder score'] if c in df.columns]
if required_cols:
    df = df.dropna(subset=required_cols)

# 8) Find 3-sigma outliers in each numeric column
#    An outlier is any value more than 3 standard deviations from the mean.
outlier_summaries = []

numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    # Use only non-missing values for the stats
    s = df[col].dropna()

    # Need a few points to measure the spread
    if len(s) < 5:
        continue

    mean = s.mean()
    std = s.std(ddof=0)  # population std
    if std == 0:
        continue

    lower = mean - 3 * std
    upper = mean + 3 * std

    # Mask rows that are out of bounds
    mask = (df[col] < lower) | (df[col] > upper)
    if mask.any():
        # Include helpful context columns
        context_cols = [c for c in ['Country name', 'Year'] if c in df.columns]
        tmp = df.loc[mask, context_cols].copy()
        tmp['column'] = col
        tmp['value'] = df.loc[mask, col]
        tmp['lower'] = lower
        tmp['upper'] = upper
        outlier_summaries.append(tmp)

# Combine all outlier rows into one DataFrame (or create an empty one if none)
if outlier_summaries:
    outliers_df = pd.concat(outlier_summaries, ignore_index=True)
else:
    base_cols = [c for c in ['Country name', 'Year'] if c in df.columns]
    #store outliers
    outliers_df = pd.DataFrame(columns=base_cols + ['column', 'value', 'lower', 'upper'])

# preview (print lines)
print("Cleaned df shape:", df.shape)
print(formatline)
print("Outliers:")
print(formatline)
print(f"{outliers_df.to_string(index=False)}\n")
print(formatline)
print("Dataframe:")
df.head(5)

Cleaned df shape: (147, 11)
---------------------------------------------------------------------------
Outliers:
---------------------------------------------------------------------------
Country name  Year                       column  value     lower    upper
 Afghanistan  2024                 Ladder score  1.364  2.115177 9.041164
   Venezuela  2024           Log GDP per capita  0.000  0.141140 2.496261
       Benin  2024               Social support  0.228  0.287115 2.379674
  Bangladesh  2024               Social support  0.286  0.287115 2.379674
 Afghanistan  2024               Social support  0.000  0.287115 2.379674
 Afghanistan  2024 Freedom to make life choices  0.000  0.218000 1.280506
   Indonesia  2024                   Generosity  0.323 -0.065267 0.292151
     Myanmar  2024                   Generosity  0.322 -0.065267 0.292151
     Finland  2024    Perceptions of corruption  0.502 -0.193653 0.487283
     Denmark  2024    Perceptions of corruption  0.488 -0.193653 0.487

,Year,Rank,Country name,Ladder score,Log GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Dystopia + residual
0,2024,1,Finland,7.736,1.749,1.783,0.824,0.986,0.110,0.502,1.782
1,2024,2,Denmark,7.521,1.825,1.748,0.820,0.955,0.150,0.488,1.535
2,2024,3,Iceland,7.515,1.799,1.840,0.873,0.971,0.201,0.173,1.659
3,2024,4,Sweden,7.345,1.783,1.698,0.889,0.952,0.170,0.467,1.385
4,2024,5,Netherlands,7.306,1.822,1.667,0.844,0.860,0.186,0.344,1.583


# Justification and reasoning for removal code above
## **1. Remove Columns Like "Unnamed: 0", "Unnamed: 1"**
>**Why:** These columns are usually leftover index columns from CSV or Excel exports. They don't contain meaningful data and can confuse later processing.

>**Necessity:** Cleaning — Removes irrelevant information, making the dataset easier to work with and less prone to errors.

## **2. Remove Columns That Are Entirely Empty:**
>**Why:** Columns where every value is missing (NaN) add no value and waste memory and time.

>**Necessity:** Filtering — Improves efficiency and avoids misleading "missing field" issues down the line.

## **3. Remove Duplicates:**
>**Why:** Duplicate rows may occur when data is appended or merged multiple times. They can bias averages, counts, or regression models.

>**Necessity:** Cleaning — Ensures each record is unique, preventing statistical distortion.

## **4. Remove Exact Duplicate Rows:**
> **Why:** Sometimes partial duplicates are removed earlier by accident. Double-running this step ensures complete deduplication and cleaner data.

> **Necessity:** Cleaning — Strengthens data integrity and is harmless if no duplicates remain.

## **5. Convert Key Numeric Columns Safely:**
>**Why:** Columns like Year, Rank, and Ladder score should be numeric for sorting, plotting, and analysis. Using errors='coerce' ensures non-numeric entries become NaN instead of breaking the code.

> **Necessity:** Transforming — Enables proper statistical and graphical analysis.

## **6. Try to Convert Other Columns to Numeric Where Appropriate:**
>**Why:** Some columns (e.g., 'GDP per capita') might look numeric but be stored as text. Attempting conversion increases consistency and allows for future numeric analyses (graph plots).

>**Necessity:** Transforming — Makes data types consistent and ready for analysis, while try/except prevents accidental loss of textual data.

## **7. Drop Rows Missing Key Fields:**
> **Why:** Rows without values in essential columns (like Year, Rank, Country name, or Ladder score) can't contribute validly to analysis.

> **Necessity:** Filtering — Ensures analyses aren't distorted by incomplete records.

## **8. Detect Outliers Using the 3-Sigma Rule**
> **Why:** The 3-sigma rule identifies extreme deviations (more than 3 standard deviations from the mean). Outliers might indicate data entry errors or exceptional cases.

> **Necessity:** Filtering/Flagging — Helps you decide whether to correct or exclude outliers, improving the reliability of the analysis.